### branch: contxt2Sentcs_wo

# Class Balance check

In [1]:
import os
import argparse
import time
import random
import shutil

from pathlib import Path
from math import ceil

from somenlp.utils import get_time_marker
from somenlp.feature_engineering import calculate_features_parallel


import os
from os import listdir 
from collections import Counter

import os
from os import listdir 
from collections import Counter


in_path = '/home/beck/Desktop/SoMeNLP/data/PLoS_methods_bio/'
out_path = '/home/beck/Desktop/SoMeNLP/data/PLoS_methods/'

def split_data(in_path, out_path, seed_, file_ext =".data.txt", 
               ratio =[60, 20, 20], set_names = ['train', 'devel', 'test'], 
               ):
    
    in_path = in_path.rstrip('/')
    out_path = out_path.rstrip('/')

    if not os.path.isdir(in_path):
        raise(RuntimeError("Input path does not exist"))
        
    if not os.path.isdir(out_path):
        os.mkdir(out_path)
    
    if sum(ratio) != 100:
        raise(RuntimeError("Input ratio {} does not sum to 100".format(ratio)))
    if len(ratio) != len(set_names):
        raise(RuntimeError("Number of ratios and setnames has to match: {} vs {}".format(ratio, set_names)))

    #print("Loading files")
    single_files = list(Path(in_path).rglob('*{}'.format(file_ext)))
    
    all_files = []
    for entry in single_files:
        
        base_file_name = entry.name.split(file_ext)[0]
        
        base_file_entries = list(Path(in_path).rglob('{}*'.format(base_file_name)))
        
        all_files.append(base_file_entries)
    
    random.seed(seed_)
    random.shuffle(all_files)

    #print("Copying files")
    cut_sum = 0
    prev_cut_idx = 0
    for cut, name in zip(ratio, set_names):
        cut_sum += cut
        cut_idx = ceil(len(all_files) * cut_sum / 100) 
        
        out_folder_name = in_path.rsplit('/')[-1]
        new_output_location = '{}/{}_{}'.format(out_path.rstrip('/'), out_folder_name, name) 
        
        if not os.path.isdir(new_output_location):
            os.makedirs(new_output_location)
            
        for files in all_files[prev_cut_idx:cut_idx]:
            for f in files:
                source_path = str(f)
                target_path = '{}/{}'.format(new_output_location, f.name)
                shutil.copy(source_path, target_path)
        prev_cut_idx = cut_idx
    
    print("Done")
    

def purposeLabel_counter(path):
    
    import os
    from os import listdir 
    from collections import Counter
    
    
    def list_file_names(path, file_ext='.labels.txt'):
    
        file_names_list = []

        for file_name in os.listdir(path):
            if not file_name.endswith(file_ext): continue
            file_names_list.append(file_name) 

        file_names_list.sort()

        return file_names_list
    
    file_name_list = list_file_names(path)
    
    interest_list = ["Analysis", "Modelling", "Stimulation", "DataCollection", "DataPreProcss", 
                 "Simulation", "Visualization", "Programming"]
    
    all_purpose_labels = []
    
    for file_name in file_name_list[:]:
        file_path = path + file_name
        
        with open(file_path, 'r') as f:
            
            list_lines = f.readlines()
            
            #print(list_of_lebs)
            list_of_tokens = ' '.join(list_lines).split()
            
            for tok in list_of_tokens:
                if tok != 'O' and (len(tok.split('-')[1].split('_')) == 3 ):
                    if tok.split('-')[1].split('_')[2] in interest_list:
                        
                        purpose = tok.split('-')[1].split('_')[2]
                        
                        all_purpose_labels.append(purpose)
    
                        
    return  dict(Counter(all_purpose_labels))

def train_test_dev(whole_path, train_path, test_path, dev_path):
    
    _whole = purposeLabel_counter(whole_path)
    _train = purposeLabel_counter(train_path)
    _test  = purposeLabel_counter(test_path)
    _dev   = purposeLabel_counter(dev_path)
    
    
    
    _whole_sorted = dict(sorted(_whole.items()))
    _train_sorted = dict(sorted(_train.items()))
    _test_sorted  = dict(sorted(_test.items()))
    _dev_sorted   = dict(sorted(_dev.items()))
    
    return _whole_sorted, _train_sorted, _test_sorted, _dev_sorted

def __proportion_calculator_with_key(whole_dict, train_dict, test_dict, dev_dict ):
    
    train_purpose_props = []
    for (k,v), (k2,v2) in zip(whole_dict.items(), train_dict.items()):
        if k2 == k: 
            train_percentage = int(v2 / v * 100)
            train_purpose_props.append((train_percentage, k))
            
    dev_purpose_props = []
    for (k,v), (k2,v2) in zip(whole_dict.items(), dev_dict.items()):
        if k2 == k:
            
            dev_percentage = int(v2 / v * 100)
            dev_purpose_props.append((dev_percentage,k))
            
    test_purpose_props = []       
    for (k,v), (k2,v2) in zip(whole_dict.items(), test_dict.items()):
        if k2 == k:
            
            test_percentage = int(v2 / v * 100)
            test_purpose_props.append((test_percentage, k))
            
    return train_purpose_props, dev_purpose_props, test_purpose_props


def __proportion_calculator(whole_dict, train_dict, test_dict, dev_dict ):
    
    train_purpose_props = []
    for (k,v), (k2,v2) in zip(whole_dict.items(), train_dict.items()):
        if k2 == k: 
            train_percentage = int(v2 / v * 100)
            train_purpose_props.append(train_percentage)
            
    dev_purpose_props = []
    for (k,v), (k2,v2) in zip(whole_dict.items(), dev_dict.items()):
        if k2 == k:
            
            dev_percentage = int(v2 / v * 100)
            dev_purpose_props.append(dev_percentage)
            
    test_purpose_props = []       
    for (k,v), (k2,v2) in zip(whole_dict.items(), test_dict.items()):
        if k2 == k:
            
            test_percentage = int(v2 / v * 100)
            test_purpose_props.append(test_percentage)
            
    return train_purpose_props, dev_purpose_props, test_purpose_props



def _within_range(list_):
    
    bool_list = []
    for val in list_:
        if val in range(55,65):
            bool_list.append(True)
        else:
            bool_list.append(False)
            
    return all(bool_list), bool_list   # apply & to all and return


def _within_range_pubmed(list_):
    
    bool_list = []
    for val in list_:
        if val in range(50,70):
            bool_list.append(True)
        else:
            bool_list.append(False)
            
            
    withn_range_bool = all(bool_list) # apply & to all and return
    
    # check if all class labels are included , class label stimulation is missing in the PubMed 
    if (len(bool_list) == 7):
        all_classes_included = True
    else:
        all_classes_included = False
        
    result = all_classes_included and withn_range_bool
            
    return result   


def _within_range_dev_test(list_):
    
    bool_list = []
    for val in list_:
        if val in range(15,25):
            bool_list.append(True)
        else:
            bool_list.append(False)
            
    return all(bool_list), bool_list   # apply & to all and return

# plos_pubmed_combined

### plos+pubmed : evaluation of split quality

In [2]:
in_path_comb = '/home/beck/Desktop/SoMeNLP/data/plos_pubmed_combined_bio/'
out_path_comb = '/home/beck/Desktop/SoMeNLP/data/plos_pubmed_combined/'

#split_data(in_path_comb, out_path_comb, 10390)


comb_path_whole = '/home/beck/Desktop/SoMeNLP/data/plos_pubmed_combined_bio/'
comb_path_train = "/home/beck/Desktop/SoMeNLP/data/plos_pubmed_combined/plos_pubmed_combined_bio_train/" 
comb_path_test = "/home/beck/Desktop/SoMeNLP/data/plos_pubmed_combined/plos_pubmed_combined_bio_test/" 
comb_path_dev = "/home/beck/Desktop/SoMeNLP/data/plos_pubmed_combined/plos_pubmed_combined_bio_devel/"
    
    
#calculate the distribution of class_labels for cmbined set
comb_whole_dict, comb_train_dict, comb_test_dict, comb_dev_dict = train_test_dev(comb_path_whole, 
                                                                                 comb_path_train, 
                                                                                 comb_path_test, 
                                                                                 comb_path_dev)


# calculate the proportion for each class label (8 - purposes)
comb_train_props, comb_dev_props, comb_test_props = __proportion_calculator(comb_whole_dict, 
                                                                            comb_train_dict, 
                                                                            comb_test_dict, 
                                                                            comb_dev_dict)

comb_train_result_ = _within_range_pubmed(comb_train_props)
comb_test_result_  = _within_range_dev_test(comb_test_props)
comb_dev_result_   = _within_range_dev_test(comb_dev_props)


print(f'plos+pubmed train {comb_train_props, _within_range(comb_train_props)[1]}')
print(f'plos+pubmed test  {comb_test_props, _within_range_dev_test(comb_test_props)[1]}')
print(f'plos+pubmed dev   {comb_dev_props, _within_range_dev_test(comb_dev_props)[1]}')
print()

print(f"plos+pubmed train within +/- 60% ? {_within_range(comb_train_props)[0]}")
print(f"plos+pubmed test  within +/- 15% ? {_within_range_dev_test(comb_test_props)[0]}")
print(f"plos+pubmed dev   within +/- 15% ? {_within_range_dev_test(comb_dev_props)[0]}")
print()

# train percentage 60%
    
for (k,v), (k2,v2) in zip(comb_whole_dict.items(),  comb_train_dict.items()):
    if k2 == k:
        print(f"{k2 :{18}}  {v2}/{v} train {v2/v*100: .2f}% ")

# test percentage 20% 
print()

for (k,v), (k2,v2) in zip(comb_whole_dict.items(),  comb_test_dict.items()):
    if k2 == k:
        print(f"{k2 :{18}}  {v2}/{v} test {v2/v*100: .2f}% ")
        
# dev percentage 20% 
print()

for (k,v), (k2,v2) in zip(comb_whole_dict.items(),  comb_dev_dict.items()):
    if k2 == k:
        print(f"{k2 :{18}}  {v2}/{v} dev {v2/v*100: .2f}% ")

plos+pubmed train ([61, 60, 61, 62, 63, 57, 63, 59], [True, True, True, True, True, True, True, True])
plos+pubmed test  ([19, 19, 19, 22, 18, 21, 19, 17], [True, True, True, True, True, True, True, True])
plos+pubmed dev   ([18, 19, 19, 15, 17, 21, 16, 23], [True, True, True, True, True, True, True, True])

plos+pubmed train within +/- 60% ? True
plos+pubmed test  within +/- 15% ? True
plos+pubmed dev   within +/- 15% ? True

Analysis            890/1436 train  61.98% 
DataCollection      150/246 train  60.98% 
DataPreProcss       205/335 train  61.19% 
Modelling           141/227 train  62.11% 
Programming         51/80 train  63.75% 
Simulation          16/28 train  57.14% 
Stimulation         45/71 train  63.38% 
Visualization       70/117 train  59.83% 

Analysis            283/1436 test  19.71% 
DataCollection      48/246 test  19.51% 
DataPreProcss       66/335 test  19.70% 
Modelling           51/227 test  22.47% 
Programming         15/80 test  18.75% 
Simulation          6/28

### Identification of files with 'Stimulation' label

This helps for manually balamcing the class labels.

In [3]:
path = "/home/beck/Desktop/SoMeNLP/data/plos_pubmed_combined/plos_pubmed_combined_bio_test/" 


def list_file_names(path, file_ext='.labels.txt'):

    file_names_list = []

    for file_name in os.listdir(path):
        if not file_name.endswith(file_ext): continue
        file_names_list.append(file_name) 

    file_names_list.sort()

    return file_names_list


# list of files names in the test set
file_name_list = list_file_names(path)


interest_list = ["Analysis", "Modelling", "Stimulation", "DataCollection", "DataPreProcss", 
             "Simulation", "Visualization", "Programming"]


for file_name in file_name_list[:]:
    file_path = path + file_name

    with open(file_path, 'r') as f:

        list_lines = f.readlines()
        list_of_tokens = ' '.join(list_lines).split()

        for tok in list_of_tokens:
            
            if tok != 'O' and (len(tok.split('-')[1].split('_')) == 3 ):
            
                if tok.split('-')[1].split('_')[2] in interest_list:

                    purpose = tok.split('-')[1].split('_')[2]

                    if 'Stimulation' in tok.split('-')[1].split('_'):
                        
                        #print(purpose, file_name)
                        print(f" {tok.split('-')[0]:{5}} {tok.split('-')[1] :{45}} {file_name}")

 B     PlugIn_Usage_Stimulation                      PMC3366939.labels.txt
 I     PlugIn_Usage_Stimulation                      PMC3366939.labels.txt
 I     PlugIn_Usage_Stimulation                      PMC3366939.labels.txt
 B     ProgrammingEnvironment_Usage_Stimulation      PMC3366939.labels.txt
 B     Application_Usage_Stimulation                 PMC3366939.labels.txt
 I     Application_Usage_Stimulation                 PMC3366939.labels.txt
 I     Application_Usage_Stimulation                 PMC3366939.labels.txt
 B     Application_Usage_Stimulation                 PMC3366939.labels.txt
 I     Application_Usage_Stimulation                 PMC3366939.labels.txt
 I     Application_Usage_Stimulation                 PMC3366939.labels.txt
 B     Application_Usage_Stimulation                 PMC3738546.labels.txt
 I     Application_Usage_Stimulation                 PMC3738546.labels.txt
 I     Application_Usage_Stimulation                 PMC3738546.labels.txt
 B     ProgrammingEnviron

`PMC3027618` and `PMC3420898` have been manually moved into dev set from test set to balance the split 100%.

### plos+pubmed iterative splitting 

In [10]:

in_path_comb = '/home/beck/Desktop/SoMeNLP/data/plos_pubmed_combined_bio/'
out_path_comb = '/home/beck/Desktop/SoMeNLP/data/plos_pubmed_combined/'


i = 203

split_data(in_path_comb, out_path_comb, i)

# combined plos pubmed bio   train , test , dev

comb_path_whole = '/home/beck/Desktop/SoMeNLP/data/plos_pubmed_combined_bio/'
comb_path_train = "/home/beck/Desktop/SoMeNLP/data/plos_pubmed_combined/plos_pubmed_combined_bio_train/" 
comb_path_test = "/home/beck/Desktop/SoMeNLP/data/plos_pubmed_combined/plos_pubmed_combined_bio_test/" 
comb_path_dev = "/home/beck/Desktop/SoMeNLP/data/plos_pubmed_combined/plos_pubmed_combined_bio_devel/"
    
    
#calculate the distribution of class_labels for cmbined set
comb_whole_dict, comb_train_dict, comb_test_dict, comb_dev_dict = train_test_dev(comb_path_whole, 
                                                                                 comb_path_train, 
                                                                                 comb_path_test, 
                                                                                 comb_path_dev)


# calculate the proportion for each class label (8 - purposes)
comb_train_props, comb_dev_props, comb_test_props = __proportion_calculator(comb_whole_dict, 
                                                                            comb_train_dict, 
                                                                            comb_test_dict, 
                                                                            comb_dev_dict)


comb_train_result_ = _within_range_pubmed(comb_train_props)
comb_test_result_  = _within_range_dev_test(comb_test_props)
comb_dev_result_   = _within_range_dev_test(comb_dev_props)

bool_ = (comb_train_result_) and (comb_test_result_) and (comb_dev_result_)



while not(bool_):
    
    print(f'splitting ... with seed: {i}')
    
    split_data(in_path_comb, out_path_comb, i)
    
    i += 1
    
    
    
    #calculate the distribution of class_labels for cmbined set
    comb_whole_dict, comb_train_dict, comb_test_dict, comb_dev_dict = train_test_dev(comb_path_whole, 
                                                                                 comb_path_train, 
                                                                                 comb_path_test, 
                                                                                 comb_path_dev)


    # calculate the proportion for each class label (8 - purposes)
    comb_train_props, comb_dev_props, comb_test_props = __proportion_calculator(comb_whole_dict, 
                                                                            comb_train_dict, 
                                                                            comb_test_dict, 
                                                                            comb_dev_dict)
    #print('is the split fair ?')
    comb_train_result_ = _within_range(comb_train_props)
    comb_test_result  = _within_range_dev_test(comb_test_props)
    comb_dev_result   = _within_range_dev_test(comb_dev_props)
    print()
    
    bool_ = (comb_train_result_) and (comb_test_result_) and (comb_dev_result_)
    

    
    
    if bool_:
        
        print('plos_pubmed split is balanced. stopped splitting')
        break
        
    else:
        
        shutil.rmtree(comb_path_train, ignore_errors=False, onerror=None)
        shutil.rmtree(comb_path_test, ignore_errors=False, onerror=None)
        shutil.rmtree(comb_path_dev, ignore_errors=False, onerror=None)
        
        print(f'result:  {comb_train_result_, comb_test_result, comb_dev_result}')   
        print(f'plos_pubmed split is not yet balanced {comb_train_props, comb_dev_props, comb_test_props}.  splitting further ...')
        print()    

Done
splitting ... with seed: 203
Done

result:  (False, False, False)
plos_pubmed split is not yet balanced ([61, 57, 50, 52, 77, 60, 64, 50], [18, 20, 27, 29, 17, 25, 9, 23], [20, 22, 22, 17, 5, 14, 25, 26]).  splitting further ...

splitting ... with seed: 204
Done

result:  (True, False, False)
plos_pubmed split is not yet balanced ([62, 63, 64, 56, 55, 60, 61, 60], [19, 20, 22, 28, 30, 7, 16, 25], [17, 15, 13, 14, 15, 32, 21, 13]).  splitting further ...

splitting ... with seed: 205
Done

result:  (False, False, False)
plos_pubmed split is not yet balanced ([57, 62, 71, 60, 82, 46, 57, 57], [19, 15, 11, 20, 7, 14, 21, 29], [22, 22, 17, 18, 10, 39, 21, 13]).  splitting further ...

splitting ... with seed: 206
Done

result:  (False, False, False)
plos_pubmed split is not yet balanced ([59, 66, 72, 58, 80, 78, 59, 58], [18, 13, 17, 11, 13, 17, 30, 21], [22, 20, 10, 29, 6, 3, 9, 20]).  splitting further ...

splitting ... with seed: 207
Done

result:  (False, False, False)
plos_pubm

Done

result:  (False, False, False)
plos_pubmed split is not yet balanced ([61, 57, 61, 67, 37, 64, 56, 50], [21, 25, 28, 20, 45, 25, 22, 29], [17, 17, 9, 12, 17, 10, 21, 19]).  splitting further ...

splitting ... with seed: 240
Done

result:  (False, False, False)
plos_pubmed split is not yet balanced ([52, 45, 47, 50, 60, 50, 46, 58], [25, 20, 37, 29, 27, 50, 32, 26], [21, 33, 15, 19, 12]).  splitting further ...

splitting ... with seed: 241
Done

result:  (False, False, False)
plos_pubmed split is not yet balanced ([56, 60, 49, 51, 66, 42, 64, 54], [20, 17, 23, 18, 26, 50, 22, 14], [22, 21, 26, 29, 7, 7, 12, 30]).  splitting further ...

splitting ... with seed: 242
Done

result:  (False, False, False)
plos_pubmed split is not yet balanced ([58, 64, 51, 51, 83, 42, 70, 63], [21, 19, 27, 24, 7, 14, 14, 19], [19, 15, 21, 24, 8, 42, 15, 17]).  splitting further ...

splitting ... with seed: 243
Done

result:  (False, True, False)
plos_pubmed split is not yet balanced ([57, 59, 50, 5

Done

result:  (False, False, False)
plos_pubmed split is not yet balanced ([64, 67, 63, 59, 45, 57, 36, 51], [15, 15, 14, 23, 41, 17, 28, 20], [20, 17, 22, 16, 13, 25, 35, 28]).  splitting further ...

splitting ... with seed: 276
Done

result:  (False, False, False)
plos_pubmed split is not yet balanced ([57, 64, 51, 56, 50, 57, 52, 64], [18, 15, 18, 15, 11, 25, 16, 9], [24, 19, 30, 27, 38, 17, 30, 26]).  splitting further ...

splitting ... with seed: 277
Done

result:  (False, True, False)
plos_pubmed split is not yet balanced ([54, 58, 54, 54, 62, 78, 67, 62], [22, 24, 20, 25, 21, 3, 11, 20], [23, 17, 24, 20, 16, 17, 21, 17]).  splitting further ...

splitting ... with seed: 278
Done

result:  (False, False, False)
plos_pubmed split is not yet balanced ([58, 56, 67, 68, 58, 64, 67, 58], [19, 22, 14, 15, 23, 17, 23, 20], [21, 21, 17, 16, 17, 17, 8, 20]).  splitting further ...

splitting ... with seed: 279
Done

result:  (False, False, False)
plos_pubmed split is not yet balanced (

Done

result:  (False, False, False)
plos_pubmed split is not yet balanced ([59, 58, 48, 55, 63, 53, 56, 67], [18, 19, 25, 19, 26, 14, 19, 15], [21, 22, 26, 25, 10, 32, 23, 17]).  splitting further ...

splitting ... with seed: 312
Done

result:  (False, False, False)
plos_pubmed split is not yet balanced ([62, 67, 67, 61, 71, 64, 52, 47], [21, 22, 22, 17, 18, 28, 14, 30], [16, 9, 9, 20, 10, 7, 33, 21]).  splitting further ...

splitting ... with seed: 313
Done

result:  (False, False, False)
plos_pubmed split is not yet balanced ([58, 64, 53, 53, 73, 50, 73, 48], [23, 18, 32, 22, 15, 10, 12, 31], [18, 17, 14, 24, 11, 39, 14, 19]).  splitting further ...

splitting ... with seed: 314
Done

result:  (False, False, False)
plos_pubmed split is not yet balanced ([65, 66, 72, 76, 50, 46, 57, 67], [17, 15, 12, 12, 12, 17, 19, 18], [16, 17, 14, 10, 37, 35, 22, 13]).  splitting further ...

splitting ... with seed: 315
Done

result:  (False, False, False)
plos_pubmed split is not yet balanced 

Done

result:  (False, False, False)
plos_pubmed split is not yet balanced ([63, 52, 56, 65, 78, 60, 63, 57], [19, 33, 22, 17, 18, 35, 30, 23], [17, 13, 21, 17, 2, 3, 5, 18]).  splitting further ...

splitting ... with seed: 348
Done

result:  (False, False, False)
plos_pubmed split is not yet balanced ([57, 54, 51, 60, 46, 57, 70, 69], [18, 24, 16, 18, 25, 21, 21, 5], [24, 21, 31, 21, 28, 21, 8, 25]).  splitting further ...

splitting ... with seed: 349
Done

result:  (False, False, False)
plos_pubmed split is not yet balanced ([61, 66, 64, 49, 47, 64, 43, 44], [18, 13, 14, 23, 27, 28, 42, 29], [20, 19, 20, 27, 25, 7, 14, 26]).  splitting further ...

splitting ... with seed: 350
Done

result:  (False, False, False)
plos_pubmed split is not yet balanced ([54, 62, 62, 52, 37, 46, 73, 71], [19, 16, 15, 17, 27, 28, 1, 14], [25, 21, 22, 29, 35, 25, 25, 13]).  splitting further ...

splitting ... with seed: 351
Done

result:  (False, False, False)
plos_pubmed split is not yet balanced ([60

...

Done

result:  (False, False, False)
plos_pubmed split is not yet balanced ([61, 64, 66, 57, 37, 82, 50, 58], [17, 18, 13, 22, 48, 14, 36, 16], [21, 16, 20, 20, 13, 3, 12, 25]).  splitting further ...

splitting ... with seed: 13560
Done

result:  (False, False, False)
plos_pubmed split is not yet balanced ([60, 60, 56, 56, 45, 67, 30, 57], [18, 21, 13, 16, 37, 3, 59, 19], [20, 18, 30, 27, 17, 28, 9, 23]).  splitting further ...

splitting ... with seed: 13561
Done

result:  (False, False, False)
plos_pubmed split is not yet balanced ([57, 66, 50, 66, 75, 100, 85, 54], [24, 10, 32, 19, 8], [18, 23, 16, 14, 16]).  splitting further ...

splitting ... with seed: 13562
Done

result:  (False, False, False)
plos_pubmed split is not yet balanced ([61, 69, 48, 48, 61, 67, 64, 64], [15, 7, 14, 11, 28, 3, 21, 7], [22, 22, 37, 40, 10, 28, 14, 27]).  splitting further ...

splitting ... with seed: 13563
Done

result:  (False, False, False)
plos_pubmed split is not yet balanced ([54, 55, 51, 56, 5

Done

result:  (False, False, False)
plos_pubmed split is not yet balanced ([62, 65, 47, 57, 53, 39, 70, 71], [17, 21, 17, 15, 40, 7, 4, 21], [19, 13, 35, 27, 6, 53, 25, 6]).  splitting further ...

splitting ... with seed: 13596
Done

result:  (False, False, False)
plos_pubmed split is not yet balanced ([63, 65, 64, 66, 56, 50, 54, 72], [15, 13, 10, 11, 22, 25, 28, 16], [20, 21, 25, 22, 21, 25, 16, 11]).  splitting further ...

splitting ... with seed: 13597
Done

result:  (False, False, False)
plos_pubmed split is not yet balanced ([58, 63, 55, 59, 73, 85, 53, 67], [23, 26, 27, 25, 5, 3, 21, 17], [17, 10, 17, 15, 21, 10, 25, 14]).  splitting further ...

splitting ... with seed: 13598
Done

result:  (False, False, False)
plos_pubmed split is not yet balanced ([58, 65, 58, 59, 57, 53, 63, 62], [21, 9, 21, 20, 7, 25, 22, 12], [20, 25, 20, 20, 35, 21, 14, 24]).  splitting further ...

splitting ... with seed: 13599
Done

result:  (False, False, False)
plos_pubmed split is not yet balanc

Done

result:  (False, False, False)
plos_pubmed split is not yet balanced ([59, 53, 68, 57, 57, 50, 35, 52], [22, 28, 16, 26, 12, 10, 32, 32], [18, 17, 14, 16, 30, 39, 32, 14]).  splitting further ...

splitting ... with seed: 13632
Done

result:  (False, False, False)
plos_pubmed split is not yet balanced ([68, 65, 71, 70, 75, 67, 56, 60], [14, 12, 16, 15, 15, 21, 26, 21], [17, 22, 11, 13, 10, 10, 16, 17]).  splitting further ...

splitting ... with seed: 13633
Done

result:  (False, False, False)
plos_pubmed split is not yet balanced ([58, 61, 61, 64, 45, 50, 63, 55], [16, 23, 16, 22, 33, 25, 23, 22], [24, 14, 22, 13, 21, 25, 12, 22]).  splitting further ...

splitting ... with seed: 13634
Done

result:  (False, False, False)
plos_pubmed split is not yet balanced ([53, 56, 48, 52, 61, 75, 57, 59], [22, 22, 14, 29, 7, 10, 22, 21], [23, 20, 36, 17, 31, 14, 19, 18]).  splitting further ...

splitting ... with seed: 13635
Done

result:  (False, False, False)
plos_pubmed split is not yet

Done

result:  (False, False, False)
plos_pubmed split is not yet balanced ([61, 56, 53, 63, 56, 57, 63, 62], [20, 17, 25, 18, 35, 28, 25, 20], [18, 26, 21, 17, 8, 14, 11, 17]).  splitting further ...

splitting ... with seed: 13668
Done

result:  (False, False, False)
plos_pubmed split is not yet balanced ([55, 69, 54, 50, 55, 67, 73, 62], [16, 11, 15, 26, 37, 17, 8, 13], [27, 18, 29, 23, 7, 14, 18, 23]).  splitting further ...

splitting ... with seed: 13669
Done

result:  (False, False, False)
plos_pubmed split is not yet balanced ([68, 70, 65, 77, 50, 78, 45, 58], [16, 11, 15, 13, 23, 3, 28, 19], [15, 17, 18, 9, 26, 17, 26, 21]).  splitting further ...

splitting ... with seed: 13670
Done

result:  (False, False, False)
plos_pubmed split is not yet balanced ([61, 57, 56, 65, 62, 71, 63, 47], [17, 16, 14, 14, 22, 17, 15, 20], [20, 26, 28, 20, 15, 10, 21, 31]).  splitting further ...

splitting ... with seed: 13671
Done

result:  (False, False, False)
plos_pubmed split is not yet bal

KeyboardInterrupt: 